In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install scikeras


In [ ]:
pip install optuna

In [ ]:
import matplotlib as mpl
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import os
import plotly.graph_objects as go
import seaborn as sns
import numpy as np
import zipfile
from datetime import timedelta
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import GRU
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
import optuna
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from keras.callbacks import Callback
from keras.regularizers import L2



In [ ]:
tromso_4135 = pd.read_csv('/content/drive/MyDrive/finaltromso.csv')
oslo_4144 =  pd.read_csv('/content/drive/MyDrive/finaloslo4144.csv')
oslo_4147 = pd.read_csv('/content/drive/MyDrive/finaloslo4147.csv')
gelio_4137 = pd.read_csv('/content/drive/MyDrive/finalgelio.csv')

In [ ]:
trondheim = pd.read_csv('/content/drive/MyDrive/7065.csv')

In [ ]:
bergen_4126 = pd.read_csv('/content/drive/MyDrive/4126.csv')

In [ ]:
bud_4152 = pd.read_csv('/content/drive/MyDrive/bud_4152_ready.csv')

In [ ]:
def label_anomalies(df, rtt_column, threshold):
    # Create a new column 'anomaly_label' where each value is 1 if the RTT is greater than the threshold, 0 otherwise
    df['anomaly_label'] = (df[rtt_column] > threshold).astype(int)
    return df

In [ ]:

def engineer_features(df, timestamp):
    # Create 'day_of_week' column with categorical day names
    df['day_of_week'] = df[timestamp].dt.day_name()

    # One-hot encode 'day_of_week' column
    df = pd.get_dummies(df, columns=['day_of_week'])

    # Drop the 'node_id' column
    df.drop('node_id', axis=1, inplace=True)


    return df


In [ ]:
def preprocess_and_subset(data, timestamp_col='ts', value_col='rtt', anomaly_threshold=0.05, start_date='2023-11-01 00:00:00', end_date='2023-11-14 23:59:59'):

    # Assuming label_anomalies is a function you've defined elsewhere that labels anomalies
    processed_data = label_anomalies(data, value_col, anomaly_threshold)

    # Convert timestamp column to datetime
    processed_data[timestamp_col] = pd.to_datetime(processed_data[timestamp_col])

    # Floor the timestamps to the nearest second
    processed_data[timestamp_col] = processed_data[timestamp_col].dt.floor(freq="s")

    processed_data = engineer_features(data, timestamp_col)

    # Set the timestamp column as the index
    processed_data = processed_data.set_index(timestamp_col)



    # Select and return the subset of data based on the specified date range
    subset = processed_data[start_date:end_date]
    return subset

In [ ]:
gelio_subset = preprocess_and_subset(gelio_4137, 'ts', 'rtt', 0.045, '2023-11-01 00:00:00', '2023-12-31 23:59:59')
tromso_subset = preprocess_and_subset(tromso_4135, 'ts', 'rtt', 0.045, '2023-11-01 00:00:00', '2023-12-31 23:59:59')
oslo4144_subset = preprocess_and_subset(oslo_4147, 'ts', 'rtt', 0.045, '2023-11-01 00:00:00', '2023-12-31 23:59:59')
oslo4147_subset = preprocess_and_subset(oslo_4144, 'ts', 'rtt', 0.045, '2023-11-01 00:00:00', '2023-12-31 23:59:59')

In [ ]:
bergen_subset = preprocess_and_subset(bergen_4126, 'ts', 'rtt', 0.045, '2023-11-01 00:00:00', '2023-12-31 23:59:59')

In [ ]:
trond_subset = preprocess_and_subset(trondheim, 'ts', 'rtt', 0.045, '2023-11-01 00:00:00', '2023-12-31 23:59:59')

In [ ]:
bud_subset = preprocess_and_subset(bud_4152, 'ts', 'rtt', 0.045, '2023-11-01 00:00:00', '2023-12-31 23:59:59')

In [ ]:
bud_subset

,rtt,event_rssi,event_rsrp,event_rsrq,event_band,event_device_state,event_lte_freq,anomaly_label,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
ts,,,,,,,,,,,,,,,
2023-11-01 00:00:00,0.051132,-80.0,-112.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
2023-11-01 00:00:01,0.050266,-80.0,-111.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
2023-11-01 00:00:02,0.048981,-80.0,-111.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
2023-11-01 00:00:03,0.047081,-80.0,-111.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
2023-11-01 00:00:04,0.047388,-80.0,-111.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 23:59:55,0.052416,-71.0,-104.0,-15.0,20.0,3.0,800.0,1,False,False,False,True,False,False,False
2023-12-31 23:59:56,0.051409,-71.0,-104.0,-15.0,20.0,3.0,800.0,1,False,False,False,True,False,False,False
2023-12-31 23:59:57,0.052315,-71.0,-104.0,-16.0,20.0,3.0,800.0,1,False,False,False,True,False,False,False


In [ ]:
gelio_subset.to_csv("/content/drive/MyDrive/gelio_feature_eng.csv", index=True)
tromso_subset.to_csv("/content/drive/MyDrive/tromso_feature_eng.csv", index=True)
oslo4147_subset.to_csv("/content/drive/MyDrive/oslo4147_feature_eng.csv", index=True)
oslo4144_subset.to_csv("/content/drive/MyDrive/oslo4144_feature_eng.csv", index=True)


In [ ]:
bud_subset.to_csv("/content/drive/MyDrive/bud_feature_eng.csv", index=True)

In [ ]:
trond_subset.to_csv("/content/drive/MyDrive/trond7065_feature_eng.csv", index=True)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout, LSTM
from keras.regularizers import L2
from keras.callbacks import EarlyStopping, TensorBoard, Callback
import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_param_importances

# Assuming data_subset is loaded here. For example:
# data_subset = pd.read_csv('your_data.csv')

class F1ScoreCallback(Callback):
    def __init__(self, val_data, val_labels):
        super().__init__()  # Initialize superclass
        self.val_data = val_data
        self.val_labels = val_labels

    def on_epoch_end(self, epoch, logs=None):
        val_pred = (self.model.predict(self.val_data) > 0.5).astype(int)
        val_f1 = f1_score(self.val_labels, val_pred, average='binary')
        print(f'\nEpoch {epoch + 1}: Validation F1 Score: {val_f1:.4f}')

# Assuming create_sequences, split_time_series_data, and preprocessing steps are correct as in your initial script

def create_model(model_type, units, dropout_rate, num_layers, l2_rate, input_shape=(100, 14)):
    model = Sequential()

    # Ensure proper handling of input_shape for all layers and include L2 regularization

    if model_type=='GRU':
      for layer_num in range(num_layers):
          if layer_num == 0:
             # For the first layer, specify the input shape
             model.add(GRU(units=units, return_sequences=True if num_layers > 1 else False,
                          dropout=dropout_rate, kernel_regularizer=L2(l2_rate),
                          input_shape=input_shape))
          else:
             # Subsequent layers do not require the input shape
             model.add(GRU(units=units, return_sequences=True if layer_num < num_layers - 1 else False,
                          dropout=dropout_rate, kernel_regularizer=L2(l2_rate)))

    else:
      for layer_num in range(num_layers):
          if layer_num == 0:
             # For the first layer, specify the input shape
             model.add(LSTM(units=units, return_sequences=True if num_layers > 1 else False,
                          dropout=dropout_rate, kernel_regularizer=L2(l2_rate),
                          input_shape=input_shape))
          else:
             # Subsequent layers do not require the input shape
             model.add(LSTM(units=units, return_sequences=True if layer_num < num_layers - 1 else False,
                          dropout=dropout_rate, kernel_regularizer=L2(l2_rate)))

    # Add remaining layers after the stacked GRUs
    model.add(Dropout(dropout_rate))
    model.add(Dense(units // 2, activation='relu', kernel_regularizer=L2(l2_rate)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=L2(l2_rate)))  # Output layer for binary classification

    # Compile the model with the specified optimizer
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_sequences(data, time_steps=100, target_column_name='anomaly_label'):
    Xs, ys = [], []
    for i in range(len(data) - time_steps):
        Xs.append(data.iloc[i:(i + time_steps)].drop(columns=[target_column_name]).values)
        ys.append(data.iloc[i + time_steps][target_column_name])
    return np.array(Xs), np.array(ys)


def split_time_series_data(Xs, ys, train_ratio=0.8, val_ratio=0.1):
    total_samples = Xs.shape[0]
    train_index = int(total_samples * train_ratio)
    val_index = train_index + int(total_samples * (val_ratio))

    X_train, y_train = Xs[:train_index], ys[:train_index]
    X_val, y_val = Xs[train_index:val_index], ys[train_index:val_index]
    X_test, y_test = Xs[val_index:], ys[val_index:]

    return X_train, y_train, X_val, y_val, X_test, y_test

def objective(trial, model_type):
    units = trial.suggest_categorical('units', [32, 64, 128])
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.3)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    l2_rate = trial.suggest_float('l2_rate', 1e-5, 1e-2, log=True)

    model = create_model(model_type, units, dropout_rate, num_layers, l2_rate)

    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    f1_callback = F1ScoreCallback(val_data=X_val_scaled, val_labels=y_val)

    model.fit(X_train_scaled, y_train, epochs=10, batch_size=batch_size,
              validation_data=(X_val_scaled, y_val), callbacks=[early_stopping, f1_callback])

    val_loss, val_accuracy = model.evaluate(X_val_scaled, y_val, verbose=0)
    predictions_proba = model.predict(X_val_scaled)
    predictions = (predictions_proba > 0.5).astype(int)
    val_f1 = f1_score(y_val, predictions, average='binary')

    trial.set_user_attr("f1_score", val_f1)
    return val_loss, val_f1


Xs, ys = create_sequences(oslo4144_subset, 100, 'anomaly_label')

X_train, y_train, X_val, y_val, X_test, y_test= split_time_series_data(Xs, ys,train_ratio=0.8, val_ratio=0.2)


scaler = StandardScaler()

# Fit on training data
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)

# Transform validation and test data
X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)


# Optimize GRU model
study_gru = optuna.create_study(directions=['minimize', 'maximize'], study_name="GRU_Optimization")
study_gru.optimize(lambda trial: objective(trial, 'GRU'), n_trials=20)

# Optimize LSTM model
study_lstm = optuna.create_study(directions=['minimize', 'maximize'], study_name="LSTM_Optimization")
study_lstm.optimize(lambda trial: objective(trial, 'LSTM'), n_trials=20)

# Example of saving and





# Saving Pareto Front for GRU
pareto_plot_gru = plot_pareto_front(study_gru, target_names=["loss", "F1"])
pareto_plot_gru.write_html("GRU_Pareto_Front.html")

# Saving Optimization History for GRU
optimization_history_gru = plot_optimization_history(study_gru, target_names=["loss", "F1"])
optimization_history_gru.write_html("GRU_Optimization_History.html")

# Saving Parameter Importances for GRU
param_importances_gru = plot_param_importances(study_gru, target_names=["loss", "F1"])
param_importances_gru.write_html("GRU_Parameter_Importances.html")


# Saving Pareto Front for LSTM
pareto_plot_lstm = plot_pareto_front(study_lstm, target_names=["loss", "F1"])
pareto_plot_lstm.write_html("LSTM_Pareto_Front.html")

# Saving Optimization History for LSTM
optimization_history_lstm = plot_optimization_history(study_lstm, target_names=["loss", "F1"])
optimization_history_lstm.write_html("LSTM_Optimization_History.html")

# Saving Parameter Importances for LSTM
param_importances_lstm = plot_param_importances(study_lstm, target_names=["loss", "F1"])
param_importances_lstm.write_html("LSTM_Parameter_Importances.html")

In [ ]:
def create_sequences(data, time_steps, target_column_name):
    Xs, ys, timestamps = [], [], []
    for i in range(len(data) - time_steps):
        # Extract features by excluding the target column
        Xs.append(data.iloc[i:(i + time_steps)].drop(columns=[target_column_name]).values)

        # Extract the target variable using the specified column name
        ys.append(data.iloc[i + time_steps][target_column_name])

        # Extract the timestamp for each sequence's target value
        timestamps.append(data.index[i + time_steps])
    return np.array(timestamps), np.array(Xs), np.array(ys)


In [ ]:
def split_time_series_data(data, Xs, ys, timestamps, train_ratio=0.7, val_ratio=0.1):

    total_rows = len(data)
    train_index = int(total_rows * train_ratio)
    val_index = train_index + int(total_rows * val_ratio)

    X_train = Xs[:train_index]
    y_train = ys[:train_index]
    timestamps_train = timestamps[:train_index]

    X_val = Xs[train_index:val_index]
    y_val = ys[train_index:val_index]
    timestamps_val = timestamps[train_index:val_index]

    X_test = Xs[val_index:]
    y_test = ys[val_index:]
    timestamps_test = timestamps[val_index:]

    return (X_train, y_train, timestamps_train,
            X_val, y_val, timestamps_val,
            X_test, y_test, timestamps_test)

In [ ]:
timestamps, Xs, ys = create_sequences(gelio_subset, 100, 'anomaly_label')

In [ ]:
X_train, y_train, timestamps_train, X_val, y_val, timestamps_val, X_test, y_test, timestamps_test = split_time_series_data(gelio_subset, Xs, ys, timestamps, train_ratio=0.7, val_ratio=0.2)


In [ ]:
scaler = StandardScaler()

# Fit on training data
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)

# Transform validation and test data
X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [ ]:
X_train_scaled

In [ ]:
def create_gru_model(units, dropout_rate, num_layers, l2_rate, input_shape=None, output_size=1, optimizer='adam'):
    model = Sequential()

    # Ensure proper handling of input_shape for all layers and include L2 regularization
    for layer_num in range(num_layers):
        if layer_num == 0:
            # For the first layer, specify the input shape
            model.add(GRU(units=units, return_sequences=True if num_layers > 1 else False,
                          dropout=dropout_rate, kernel_regularizer=L2(l2_rate),
                          input_shape=input_shape))
        else:
            # Subsequent layers do not require the input shape
            model.add(GRU(units=units, return_sequences=True if layer_num < num_layers - 1 else False,
                          dropout=dropout_rate, kernel_regularizer=L2(l2_rate)))

    # Add remaining layers after the stacked GRUs
    model.add(Dropout(dropout_rate))
    model.add(Dense(units // 2, activation='relu', kernel_regularizer=L2(l2_rate)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(output_size, activation='sigmoid', kernel_regularizer=L2(l2_rate)))  # Output layer for binary classification

    # Compile the model with the specified optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model



#model = KerasClassifier(model=create_gru_model, model__input_shape=(100, 8), model__units=64, model__dropout_rate=0.2, model__optimizer='adam', model__output_size=1)


In [ ]:
def create_LSTM_model(units, dropout_rate, num_layers, input_shape=None, output_size=1, optimizer='adam'):

  model = Sequential()

  # Ensure proper handling of input_shape for all layers
  for _ in range(num_layers):
    model.add(LSTM(units=units, return_sequences=True, dropout=dropout_rate,
                  input_shape=input_shape if _ == 0 else (None, input_shape[-1])))

  # Add remaining layers after the stacked GRUs
  model.add(LSTM(units, return_sequences=False))  # Final GRU layer with return_sequences=False
  model.add(Dropout(dropout_rate))
  model.add(Dense(units // 2, activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(output_size, activation='sigmoid'))  # Output layer for binary classification

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
f1_callback = F1ScoreCallback(X_train, y_train, X_val_scaled, y_val)


In [ ]:
def objective(trial):
  # Define hyperparameters using trial object
  units = trial.suggest_categorical('units', [32, 64, 128])
  dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.3)
  num_layers = trial.suggest_int('num_layers', 1, 3)  # Search for 1 to 3 layers
  batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
  l2_rate = trial.suggest_float('l2_rate', 1e-5, 1e-2, log=True)
  input_shape = (100, 14)

  # Create GRU model with additional layers
  model = create_gru_model(units=units, dropout_rate=dropout_rate, num_layers=num_layers,
                           input_shape= input_shape, l2_rate=l2_rate)

  # Early stopping callback
  early_stopping = EarlyStopping(monitor='val_loss', patience=4)


  # TensorBoard callback for visualization (optional)
  tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/logs', write_graph=True)  # Adjust log directory as needed

  # Model fitting with callbacks
  history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=batch_size,
                      validation_data=(X_val_scaled, y_val),
                      callbacks=[early_stopping, f1_callback])

  # Evaluation based on test set
  test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

  # Use predictions on the validation set for F1 score calculation
  predictions_proba = model.predict(X_val_scaled)
  predictions = (predictions_proba > 0.5).astype(int)

  f1 = f1_score(y_val, predictions, average='binary')  # Note: Changed to y_val for consistency with validation phase

  # Logging additional metrics for analysis
  trial.set_user_attr("f1_score", f1)

  # Optimize for test loss, but monitor F1 score
  return test_loss, f1

In [ ]:
study = optuna.create_study(directions=['minimize', 'maximize'],
                            study_name="GRU1")

# Optimize the study, the study object is updated in place
study.optimize(objective, n_trials=20)



In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import f1_score

class F1ScoreCallback(Callback):
    def __init__(self, train_data, train_labels, val_data, val_labels):
        self.train_data = train_data
        self.train_labels = train_labels
        self.val_data = val_data
        self.val_labels = val_labels
        self.val_f1_scores = []  # Optionally store validation F1 scores

    def on_epoch_end(self, epoch, logs=None):
        # Efficient prediction and F1 score computation
        val_pred = (self.model.predict(self.val_data) > 0.5).astype(int)
        val_f1 = f1_score(self.val_labels, val_pred, average='binary')
        self.val_f1_scores.append(val_f1)  # Store if needed
        print(f'\nEpoch {epoch+1}: Validation F1 Score: {val_f1:.4f}')


In [ ]:

# Set the learning rate
learning_rate = 0.001  # Example learning rate

# Create the optimizer with the desired learning rate
adam_optimizer = Adam(learning_rate=learning_rate)

l2_regularizer_rate = 0.001  # This is a hyperparameter you can tune

model = Sequential()
model.add(GRU(units=128, return_sequences=False, dropout=0.2,
              input_shape=(100, 14),
              kernel_regularizer=L2(l2_regularizer_rate),  # Apply L2 regularization to the GRU layer
              recurrent_regularizer=L2(l2_regularizer_rate)))  # Optional: Apply to recurrent kernel as well
model.add(Dropout(0.2))
model.add(Dense(128 // 2, activation='relu',
                kernel_regularizer=L2(l2_regularizer_rate)))  # Apply L2 regularization to the Dense layer
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid',
                kernel_regularizer=L2(l2_regularizer_rate)))  # Apply L2 regularization to the output layer

model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=2)


  # TensorBoard callback for visualization (optional)
#tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/logs', write_graph=True)  # Adjust log directory as needed

history =  model.fit(X_train_scaled, y_train, epochs=50, batch_size=32,
                        validation_data=(X_val_scaled, y_val),
                        callbacks=[early_stopping, f1_callback])

  # Evaluation
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

  # You can report additional metrics with trial.set_user_attr
predictions_proba = model.predict(X_test_scaled)
predictions = (predictions_proba > 0.5).astype(int)  # Thresholding to get binary predictions

  # Calculate metrics
f1 = f1_score(y_test, predictions, average='macro')
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)

In [ ]:
oslo4144_subset

In [ ]:
precision

0.7514765938776199

In [ ]:
best_trial = study.best_trials

In [ ]:
best_trial

[FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.4322155714035034, 0.46587222790442795], datetime_start=datetime.datetime(2024, 3, 17, 20, 48, 54, 10396), datetime_complete=datetime.datetime(2024, 3, 17, 21, 13, 28, 880122), params={'units': 128, 'dropout_rate': 0.2715982395549798, 'num_layers': 1, 'batch_size': 32, 'epochs': 100}, user_attrs={'f1_score': 0.46587222790442795, 'precision': 0.5434782608695652, 'recall': 0.0030120481927710845}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'units': CategoricalDistribution(choices=(32, 64, 128)), 'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.2, step=None), 'num_layers': IntDistribution(high=3, log=False, low=1, step=1), 'batch_size': CategoricalDistribution(choices=(32, 64, 128)), 'epochs': CategoricalDistribution(choices=(25, 50, 100, 150))}, trial_id=1, value=None)]

In [ ]:
df = study.trials_dataframe()
df.to_csv("study_results.csv", index=False)

In [ ]:
import optuna
print(optuna.__version__)


In [ ]:
# Access the best hyperparameters and potentially the fine-tuned model
optuna.visualization.plot_pareto_front(study, target_names=["loss", "F1"])


In [ ]:
#study = optuna.load_study('your_study_name.db')  # Assuming the study is saved

# Get trial information
trials = study.trials

# Example: Plot validation loss of all trials
import matplotlib.pyplot as plt

losses = [trial.value for trial in trials]
plt.plot(losses)
plt.xlabel("Trial")
plt.ylabel("Validation Loss")
plt.show()

In [ ]:
input_shape = (100, 8)  # 100 time steps, 10 features per step
model = create_gru_model(input_shape, units=64, dropout_rate=0.2, output_size=1)


In [ ]:
def build_model_LSTM_AE(sequence_length, num_features, lstm_units, dropout_rate):
    model = Sequential([
        LSTM(lstm_units, input_shape=(sequence_length, num_features), return_sequences=True),
        Dropout(dropout_rate),
        LSTM(int(lstm_units / 2), return_sequences=False),
        Dropout(dropout_rate),
        RepeatVector(sequence_length),
        LSTM(int(lstm_units / 2), return_sequences=True),
        LSTM(lstm_units, return_sequences=True),
        TimeDistributed(Dense(num_features))
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model


In [ ]:
lstm_units_options = [64, 128]
dropout_options = [0.2, 0.3]

In [ ]:
best_loss = np.inf
best_config = {}

for units in lstm_units_options:
    for dropout in dropout_options:
        # Adjust your data preprocessing and sequence creation as needed
        model = build_model(sequence_length, num_features, units, dropout)

        # Assuming you have a function to prepare your data: sequences_train, sequences_val
        history = model.fit(sequences_train, sequences_train,
                            epochs=100,
                            batch_size=64,  # You might also iterate over batch size options
                            validation_data=(sequences_val, sequences_val),
                            callbacks=[EarlyStopping(monitor='val_loss', patience=5)],
                            verbose=0)

        val_loss = min(history.history['val_loss'])
        if val_loss < best_loss:
            best_loss = val_loss
            best_config = {'units': units, 'dropout': dropout}

print("Best configuration:", best_config)